In [1]:
from common_import import *
import prettytable
result_dict = readpkl('result_dict.pkl')

In [64]:
def print_and_save_result(result_dict, checkpoint_list, output_file, save_excel=True):
    import prettytable

    # PrettyTable 객체 생성 및 필드 설정
    pt = prettytable.PrettyTable()
    pt.field_names = ['Dataset', 'E1', 'E2', 'Checkpoint', 'Subset']


    # 데이터를 리스트에 수집
    data = []
    for key in checkpoint_list:
        if key not in result_dict.keys():
            print(f"Checkpoint {key}에 대한 결과가 없습니다.")
            continue
        for subset in result_dict[key].keys():
            #if 'WITH_RZ' in subset: continue
            if 'STEP' in subset: continue
            if 'TRAIN_UNIV' in subset: continue
            splited = subset.split('-')
            if 'CAM_NO_FACTOR' in splited[-1] or 'CANONICAL' in splited[-1]:
                dataset = splited[0]
            else:
                dataset = splited[0] + '-' + splited[-1]
            #print(dataset, type(dataset))
            e1 = float(result_dict[key][subset]['e1'])
            e2 = float(result_dict[key][subset]['e2'])
            data.append([dataset, e1, e2, key, subset])

    # 데이터를 Subset 1순위, checkpoint 2순위로 정렬
    data_sorted = sorted(data, key=lambda x: (x[0], len(x[3])))

    prev_dataset = ''
    for i, row in enumerate(data_sorted):
        if prev_dataset != '' and prev_dataset != row[0]:
            data_sorted[i-1].append(True)
        else:
            data_sorted[i-1].append(False)
        prev_dataset = row[0]

    # 정렬된 데이터를 테이블에 추가
    for row in data_sorted:
        pt.add_row([row[0], f'{row[1]:.2f}', f'{row[2]:.2f}', row[3], row[4]], divider=row[-1])

    # 테이블 출력
    print(pt)

    # 엑셀로 저장
    if save_excel:
        df = pd.DataFrame(pt.rows, columns=pt.field_names)
        df.to_excel(output_file, index=False)
        print(f"PrettyTable 데이터를 {output_file}로 저장했습니다!")

def seed_summary(result_dict, checkpoint_list):
    field_names_e1 = []
    field_names_e2 = []

    # 데이터를 리스트에 수집
    data = {}
    seed_list = []
    for key in checkpoint_list:
        if key not in result_dict.keys():
            print(f"Checkpoint {key}에 대한 결과가 없습니다.")
            continue
        if 'seed1' in key: seed = 1
        elif 'seed2' in key: seed = 2
        elif 'seed3' in key: seed = 3
        elif 'seed4' in key: seed = 4
        else: seed = 0
        seed_list.append(seed)
        field_names_e1.append(f'seed{seed}_e1')
        field_names_e2.append(f'seed{seed}_e2')
        for subset in result_dict[key].keys():
            #if 'CANONICAL_PCL' in subset: continue
            if 'ROT' in subset: continue
            if 'TRAIN_UNIV' in subset: continue
            if subset not in data.keys():
                data[subset] = {}
            data[subset][seed] = {}
            data[subset][seed]['e1'] = float(result_dict[key][subset]['e1'])
            data[subset][seed]['e2'] = float(result_dict[key][subset]['e2'])
    field_names_e1.append('Average_e1')
    field_names_e2.append('Average_e2')
    #print(seed_list)

    # PrettyTable 객체 생성 및 필드 설정
    pt = prettytable.PrettyTable()
    pt.field_names = ['Subset'] + field_names_e1 + field_names_e2 # ['Subset', 'seed0_e1', 'seed1_e1', 'seed2_e1', 'seed3_e1', 'seed4_e1', 'Average_e1', 'seed0_e2', 'seed1_e2', 'seed2_e2', 'seed3_e2', 'seed4_e2', 'Average_e2']
    #print(pt.field_names)
    #print(data.keys())
    
    rows = []
    for subset in data.keys():
        #print(subset)
        if   subset == '3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN'   : row = ['3DHP TRAIN']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6'       : row = ['3DHP TEST']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6_UNIV'  : row = ['3DHP TEST (univ)']
        elif subset == 'FIT3D-GT-CAM_NO_FACTOR-ALL_TEST'        : row = ['FIT3D ALL']
        elif subset == 'H36M-GT-CAM_NO_FACTOR'                  : row = ['H36M S9,11']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-TR_S1_TS_S5678'   : row = ['H36M S5678']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_ALL_TRAIN'  : row = ['3DHP TRAIN']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_TS1_6'      : row = ['3DHP TEST']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_TS1_6_UNIV' : row = ['3DHP TEST (univ)']
        elif subset == 'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-ALL_TEST'       : row = ['FIT3D ALL']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ'                 : row = ['H36M S9,11']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TR_S1_TS_S5678'  : row = ['H36M S5678']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_ALL_TRAIN'  : row = ['3DHP TRAIN']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_TS1_6'      : row = ['3DHP TEST']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_TS1_6_UNIV' : row = ['3DHP TEST (univ)']
        elif subset == 'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-ALL_TEST'       : row = ['FIT3D ALL']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE'                 : row = ['H36M S9,11']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TR_S1_TS_S5678'  : row = ['H36M S5678']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_WITH_KVIRT-TEST_ALL_TRAIN'  : row = ['3DHP TRAIN']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_WITH_KVIRT-TEST_TS1_6'      : row = ['3DHP TEST']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_WITH_KVIRT-TEST_TS1_6_UNIV' : row = ['3DHP TEST (univ)']
        elif subset == 'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_WITH_KVIRT-ALL_TEST'       : row = ['FIT3D ALL']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_WITH_KVIRT'                 : row = ['H36M S9,11']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_WITH_KVIRT-TR_S1_TS_S5678'  : row = ['H36M S5678']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_NO_RZ-TEST_ALL_TRAIN'  : row = ['3DHP TRAIN']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_NO_RZ-TEST_TS1_6'      : row = ['3DHP TEST']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_NO_RZ-TEST_TS1_6_UNIV' : row = ['3DHP TEST (univ)']
        elif subset == 'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_NO_RZ-ALL_TEST'       : row = ['FIT3D ALL']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_NO_RZ'                 : row = ['H36M S9,11']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_NO_RZ-TR_S1_TS_S5678'  : row = ['H36M S5678']
        else: row = [subset]
        #print(data[subset].keys())
        for seed in seed_list:
            #print(seed)
            row.append(data[subset][seed]['e1'])
        row.append(np.mean([data[subset][i]['e1'] for i in seed_list]))
        for seed in seed_list:
            row.append(data[subset][seed]['e2'])
        row.append(np.mean([data[subset][i]['e2'] for i in seed_list]))
        rows.append(row)
        #rows.append([subset+'_e1', data[subset][0]['e1'], data[subset][1]['e1'], data[subset][2]['e1'], data[subset][3]['e1'], data[subset][4]['e1'], np.mean([data[subset][i]['e1'] for i in range(5)]), data[subset][0]['e2'], data[subset][1]['e2'], data[subset][2]['e2'], data[subset][3]['e2'], data[subset][4]['e2'], np.mean([data[subset][i]['e2'] for i in range(5)])])

    rows = sorted(rows, key=lambda x: (x[0]))

    # 정렬된 데이터를 테이블에 추가
    for row in rows:
        temp = [row[0]]
        temp += [f'{row[i]:.2f}' for i in range(1, len(row))]
        pt.add_row(temp)
    print(pt)
    return pt

# Compare Models

### Trained on H36M S15678 

In [7]:
print(*result_dict['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering'].keys(), sep='\n')

3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_TS1_6_UNIV
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_TS1_6
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_TS1_6
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_TS1_6_UNIV
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-ALL_TEST
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_ALL_TRAIN
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_ALL_TRAIN
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-ALL_TEST


In [14]:
delete_list = []
for key in result_dict['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering'].keys():
    if 'PCL' not in key: continue
    delete_list.append(key)
    print(key)

for key in delete_list:
    del result_dict['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering'][key]
    
savepkl(result_dict, 'result_dict.pkl')

In [37]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering{seed}',
        f'MB_train_h36m_gt_cam_no_factor{seed}',
        f'MB_train_h36m_gt_cam_no_factor_input_centering{seed}',
        #f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original{seed}',
    ]
    output_file = f"output_h36m{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file, save_excel=True)
    print()

+----------------------+--------+--------+---------------------------------------------------------------------------------+------------------------------------------------------------------------+
|       Dataset        |   E1   |   E2   |                                    Checkpoint                                   |                                 Subset                                 |
+----------------------+--------+--------+---------------------------------------------------------------------------------+------------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN  | 95.59  | 68.06  |                          MB_train_h36m_gt_cam_no_factor                         |                  3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN                  |
| 3DHP-TEST_ALL_TRAIN  | 115.18 | 58.73  |                  MB_train_h36m_gt_cam_no_factor_input_centering                 |                  3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN                  |
| 3DHP-TES

### H36M S1 TRAIN

In [39]:
delete_list = []
for key in result_dict['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678'].keys():
    if 'PCL' not in key: continue
    delete_list.append(key)
    print(key)

for key in delete_list:
    del result_dict['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678'][key]
    
savepkl(result_dict, 'result_dict.pkl')

3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_TS1_6_UNIV
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TR_S1_TS_S5678
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_TS1_6
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_ALL_TRAIN
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-ALL_TEST


In [41]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678{seed}',
        f'MB_train_h36m_gt_cam_no_factor_input_centering_tr_s1_ts_s5678{seed}',
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678{seed}',
        #f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s1_ts_s5678{seed}',
        #f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering_tr_s1_ts_s5678{seed}',
    ]
    output_file = f"output_h36m_tr_s1_ts_s5678{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file, save_excel=True)
    print()

+----------------------+--------+-------+------------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|       Dataset        |   E1   |   E2  |                                           Checkpoint                                           |                                 Subset                                 |
+----------------------+--------+-------+------------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN  | 116.92 | 77.36 |                         MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678                          |                  3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN                  |
| 3DHP-TEST_ALL_TRAIN  | 122.77 | 66.04 |                 MB_train_h36m_gt_cam_no_factor_input_centering_tr_s1_ts_s5678                  |              

### FIT3D TR_S03

In [43]:
delete_list = []
for key in result_dict['MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03'].keys():
    if 'PCL' not in key: continue
    delete_list.append(key)
    print(key)

for key in delete_list:
    del result_dict['MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03'][key]
    
savepkl(result_dict, 'result_dict.pkl')

H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TR_S1_TS_S5678
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TR_S03
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_TS1_6
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_ALL_TRAIN


In [44]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_fit3d_gt_cam_no_factor_tr_s03{seed}',
        #f'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s03{seed}',
        f'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03{seed}',
    ]
    print(f"**** seed = {seed} ****")
    output_file = f"output_fit3d_tr_s03{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file)
    print()

**** seed =  ****
+----------------------+--------+--------+-----------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|       Dataset        |   E1   |   E2   |                                        Checkpoint                                       |                                 Subset                                 |
+----------------------+--------+--------+-----------------------------------------------------------------------------------------+------------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN  | 216.50 | 144.11 |                          MB_train_fit3d_gt_cam_no_factor_tr_s03                         |                  3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN                  |
| 3DHP-TEST_ALL_TRAIN  | 178.32 | 128.28 | MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03 | 3DHP-GT-CAM_NO_FACTOR-INP

### FIT3D TS_S4710

In [46]:
delete_list = []
for key in result_dict['MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710'].keys():
    if 'PCL' not in key: continue
    delete_list.append(key)
    print(key)

for key in delete_list:
    del result_dict['MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710'][key]
    
savepkl(result_dict, 'result_dict.pkl')

H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TR_S1_TS_S5678
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_TS1_6
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_ALL_TRAIN
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TS_S4710


In [47]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_fit3d_gt_cam_no_factor_ts_s4710{seed}',
        #f'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_ts_s4710{seed}',
        f'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710{seed}',
    ]
    output_file = f"output_fit3d_ts_s4710{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file)
    print()

+----------------------+--------+--------+-------------------------------------------------------------------------------------------+---------------------------------------------------------------------+
|       Dataset        |   E1   |   E2   |                                         Checkpoint                                        |                                Subset                               |
+----------------------+--------+--------+-------------------------------------------------------------------------------------------+---------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN  | 205.57 | 137.99 |                          MB_train_fit3d_gt_cam_no_factor_ts_s4710                         |                 3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN                |
| 3DHP-TEST_ALL_TRAIN  | 165.81 | 114.84 | MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710 |  3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_R

### 3DHP S1

In [48]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_3dhp_gt_cam_no_factor_tr_s1{seed}',
        #f'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s1{seed}',
        f'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1{seed}',
    ]
    print(f"**** seed = {seed} ****")
    output_file = f"output_3dhp_tr_s1{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file)
    print()

**** seed =  ****
+----------------------+--------+--------+---------------------------------------------------------------------------------------+---------------------------------------------------------------------+
|       Dataset        |   E1   |   E2   |                                       Checkpoint                                      |                                Subset                               |
+----------------------+--------+--------+---------------------------------------------------------------------------------------+---------------------------------------------------------------------+
|   3DHP-TEST_TS1_6    | 69.46  | 47.56  |                          MB_train_3dhp_gt_cam_no_factor_tr_s1                         |                   3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6                  |
|   3DHP-TEST_TS1_6    | 66.44  | 47.83  | MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1 |    3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_R

# Seed Summary

## Trained on H36M S15678

### baseline

In [50]:
checkpoint_list = []
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list += [
        f'MB_train_h36m_gt_cam_no_factor{seed}',
    ]
pt = seed_summary(result_dict, checkpoint_list)

# save to excel
df = pd.DataFrame(pt.rows, columns=pt.field_names)
df.to_excel('output_h36M_s15678_baseline.xlsx', index=False)

+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+
|      Subset      | seed0_e1 | seed1_e1 | seed2_e1 | seed3_e1 | seed4_e1 | Average_e1 | seed0_e2 | seed1_e2 | seed2_e2 | seed3_e2 | seed4_e2 | Average_e2 |
+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+
|    3DHP TEST     |  97.87   |  86.18   |  93.50   |  92.77   |  95.96   |   93.26    |  61.34   |  56.70   |  58.65   |  58.62   |  60.82   |   59.23    |
| 3DHP TEST (univ) |  71.88   |  64.87   |  68.99   |  69.56   |  72.41   |   69.54    |  54.68   |  50.60   |  52.30   |  52.34   |  54.26   |   52.84    |
|    3DHP TRAIN    |  95.59   |  89.68   |  91.36   |  99.12   |  93.26   |   93.80    |  68.06   |  65.72   |  64.20   |  72.77   |  67.72   |   67.69    |
|    FIT3D ALL     |  187.59  |  166.97  |  170.89  |  159

### input_centering

In [52]:
checkpoint_list = []
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list += [
        f'MB_train_h36m_gt_cam_no_factor_input_centering{seed}',
    ]
pt = seed_summary(result_dict, checkpoint_list)

# save to excel
df = pd.DataFrame(pt.rows, columns=pt.field_names)
df.to_excel('output_h36M_s15678_input_centering.xlsx', index=False)

+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+
|      Subset      | seed0_e1 | seed1_e1 | seed2_e1 | seed3_e1 | seed4_e1 | Average_e1 | seed0_e2 | seed1_e2 | seed2_e2 | seed3_e2 | seed4_e2 | Average_e2 |
+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+
|    3DHP TEST     |  107.49  |  108.54  |  113.40  |  110.81  |  104.77  |   109.00   |  55.61   |  57.44   |  58.90   |  58.97   |  54.98   |   57.18    |
| 3DHP TEST (univ) |  89.16   |  88.80   |  90.18   |  95.02   |  84.54   |   89.54    |  49.59   |  51.22   |  52.57   |  52.58   |  49.10   |   51.01    |
|    3DHP TRAIN    |  115.18  |  115.14  |  112.78  |  118.35  |  110.09  |   114.31   |  58.73   |  58.46   |  57.70   |  60.02   |  56.89   |   58.36    |
|    FIT3D ALL     |  100.05  |  130.99  |  92.68   |  133

### canonical_pcl_with_Rz_input_centering

In [53]:
checkpoint_list = []
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list += [
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_with_Rz_input_centering{seed}',
    ]
pt = seed_summary(result_dict, checkpoint_list)

# save to excel
df = pd.DataFrame(pt.rows, columns=pt.field_names)
df.to_excel('output_h36M_s15678_canonical_pcl_with_Rz_input_centering.xlsx', index=False)

Checkpoint MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_with_Rz_input_centering_seed3에 대한 결과가 없습니다.
+------------------+----------+----------+----------+----------+------------+----------+----------+----------+----------+------------+
|      Subset      | seed0_e1 | seed1_e1 | seed2_e1 | seed4_e1 | Average_e1 | seed0_e2 | seed1_e2 | seed2_e2 | seed4_e2 | Average_e2 |
+------------------+----------+----------+----------+----------+------------+----------+----------+----------+----------+------------+
|    3DHP TEST     |  88.93   |  96.77   |  80.22   |  79.62   |   86.38    |  57.96   |  64.59   |  53.49   |  57.12   |   58.29    |
| 3DHP TEST (univ) |  70.48   |  74.92   |  60.27   |  66.12   |   67.95    |  51.71   |  57.65   |  47.71   |  50.97   |   52.01    |
|    3DHP TRAIN    |  81.27   |  88.58   |  72.86   |  75.36   |   79.52    |  61.96   |  69.60   |  56.81   |  61.22   |   62.40    |
|    FIT3D ALL     |  91.76   |  117.54  |  101.54  |  98.03   |   102.22   |  

### canonical_3d_revolute_with_Kvirt_input_centering

In [54]:
checkpoint_list = []
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list += [
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_with_Kvirt_input_centering{seed}',
    ]
pt = seed_summary(result_dict, checkpoint_list)

# save to excel
df = pd.DataFrame(pt.rows, columns=pt.field_names)
df.to_excel('output_h36m_s15678_canonical_revolute_with_Kvirt_input_centering.xlsx', index=False)

+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+
|      Subset      | seed0_e1 | seed1_e1 | seed2_e1 | seed3_e1 | seed4_e1 | Average_e1 | seed0_e2 | seed1_e2 | seed2_e2 | seed3_e2 | seed4_e2 | Average_e2 |
+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+
|    3DHP TEST     |  157.56  |  159.38  |  155.98  |  157.85  |  150.48  |   156.25   |  95.20   |  85.14   |  96.51   |  100.82  |  86.30   |   92.80    |
| 3DHP TEST (univ) |  117.41  |  116.96  |  120.14  |  118.59  |  110.18  |   116.65   |  84.82   |  75.87   |  86.08   |  89.84   |  76.96   |   82.71    |
|    3DHP TRAIN    |  136.97  |  139.24  |  135.84  |  140.10  |  131.75  |   136.78   |  90.02   |  85.36   |  95.73   |  97.21   |  83.84   |   90.43    |
|    FIT3D ALL     |  171.07  |  148.82  |  135.01  |  160

### canonical_3d_revolute_no_Rz_input_centering

In [61]:
delete_list = []
for key in result_dict['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering'].keys():
    if 'PCL' not in key: continue
    delete_list.append(key)
    print(key)

for key in delete_list:
    del result_dict['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering'][key]
    
savepkl(result_dict, 'result_dict.pkl')

3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL-TEST_TS1_6
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL-TEST_TS1_6_UNIV
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL-TEST_ALL_TRAIN
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_PCL-ALL_TEST
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL


In [66]:
checkpoint_list = []
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list += [
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering{seed}',
    ]
#print(checkpoint_list)
pt = seed_summary(result_dict, checkpoint_list)

# save to excel
df = pd.DataFrame(pt.rows, columns=pt.field_names)
df.to_excel('output_h36m_s15678_canonical_revolute_no_Rz_input_centering.xlsx', index=False)

+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+
|      Subset      | seed0_e1 | seed1_e1 | seed2_e1 | seed3_e1 | seed4_e1 | Average_e1 | seed0_e2 | seed1_e2 | seed2_e2 | seed3_e2 | seed4_e2 | Average_e2 |
+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+
|    3DHP TEST     |  99.48   |  80.48   |  83.80   |  93.36   |  79.42   |   87.31    |  65.42   |  55.07   |  58.82   |  62.58   |  55.08   |   59.40    |
| 3DHP TEST (univ) |  79.04   |  60.40   |  65.05   |  74.08   |  64.04   |   68.52    |  58.35   |  49.13   |  52.49   |  55.83   |  49.17   |   52.99    |
|    3DHP TRAIN    |  94.57   |  72.83   |  78.77   |  87.34   |  75.53   |   81.81    |  69.24   |  58.92   |  63.04   |  67.09   |  60.84   |   63.83    |
|    FIT3D ALL     |  102.73  |  111.50  |  95.90   |  91.

### canonical_3d_revolute_input_centering

In [67]:
checkpoint_list = []
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list += [
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering{seed}',
    ]
#print(checkpoint_list)
pt = seed_summary(result_dict, checkpoint_list)

# save to excel
df = pd.DataFrame(pt.rows, columns=pt.field_names)
df.to_excel('output_h36m_s15678_canonical_revolute_input_centering.xlsx', index=False)

+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+
|      Subset      | seed0_e1 | seed1_e1 | seed2_e1 | seed3_e1 | seed4_e1 | Average_e1 | seed0_e2 | seed1_e2 | seed2_e2 | seed3_e2 | seed4_e2 | Average_e2 |
+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+
|    3DHP TEST     |  79.36   |  79.67   |  90.07   |  95.64   |  89.80   |   86.91    |  53.81   |  53.55   |  58.20   |  62.35   |  61.00   |   57.78    |
| 3DHP TEST (univ) |  61.18   |  59.53   |  65.89   |  76.49   |  69.97   |   66.61    |  48.03   |  47.79   |  51.93   |  55.65   |  54.44   |   51.57    |
|    3DHP TRAIN    |  70.31   |  72.88   |  80.99   |  90.05   |  81.68   |   79.18    |  56.37   |  57.27   |  62.35   |  68.03   |  62.85   |   61.37    |
|    FIT3D ALL     |  87.64   |  138.49  |  92.71   |  91.

## Trained on H36M S1

### baseline

In [81]:
checkpoint_list = []
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list += [
        f'MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678{seed}',
    ]
#print(checkpoint_list)
pt = seed_summary(result_dict, checkpoint_list)

# save to excel
df = pd.DataFrame(pt.rows, columns=pt.field_names)
df.to_excel('output_h36m_s1_baseline.xlsx', index=False)

+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+
|      Subset      | seed0_e1 | seed1_e1 | seed2_e1 | seed3_e1 | seed4_e1 | Average_e1 | seed0_e2 | seed1_e2 | seed2_e2 | seed3_e2 | seed4_e2 | Average_e2 |
+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+
|    3DHP TEST     |  111.85  |  111.78  |  113.90  |  116.84  |  113.04  |   113.48   |  72.21   |  71.51   |  73.64   |  75.02   |  72.11   |   72.90    |
| 3DHP TEST (univ) |  89.05   |  87.85   |  90.86   |  92.97   |  87.75   |   89.69    |  64.47   |  63.85   |  65.72   |  66.95   |  64.36   |   65.07    |
|    3DHP TRAIN    |  116.92  |  118.65  |  116.27  |  118.06  |  114.71  |   116.92   |  77.36   |  77.80   |  76.38   |  77.43   |  75.52   |   76.90    |
|    FIT3D ALL     |  158.49  |  173.20  |  185.30  |  179

### input centering

In [80]:
checkpoint_list = []
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list += [
        f'MB_train_h36m_gt_cam_no_factor_input_centering_tr_s1_ts_s5678{seed}',
    ]
#print(checkpoint_list)
pt = seed_summary(result_dict, checkpoint_list)

# save to excel
df = pd.DataFrame(pt.rows, columns=pt.field_names)
df.to_excel('output_h36m_s1_input_centering.xlsx', index=False)

Checkpoint MB_train_h36m_gt_cam_no_factor_input_centering_tr_s1_ts_s5678_seed4에 대한 결과가 없습니다.
+------------------+----------+----------+----------+----------+------------+----------+----------+----------+----------+------------+
|      Subset      | seed0_e1 | seed1_e1 | seed2_e1 | seed3_e1 | Average_e1 | seed0_e2 | seed1_e2 | seed2_e2 | seed3_e2 | Average_e2 |
+------------------+----------+----------+----------+----------+------------+----------+----------+----------+----------+------------+
|    3DHP TEST     |  117.87  |  122.64  |  123.06  |  119.13  |   120.67   |  66.00   |  68.07   |  69.86   |  68.84   |   68.19    |
| 3DHP TEST (univ) |  96.63   |  100.04  |  102.25  |  98.08   |   99.25    |  58.89   |  60.73   |  62.33   |  61.42   |   60.84    |
|    3DHP TRAIN    |  122.77  |  123.94  |  125.40  |  121.90  |   123.50   |  66.04   |  67.47   |  67.15   |  66.15   |   66.70    |
|    FIT3D ALL     |  130.44  |  160.44  |  130.05  |  152.35  |   143.32   |  81.92   |  97.32  

### canonical_3d_revolute_input_centering

In [78]:
checkpoint_list = []
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list += [
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678{seed}',
    ]
#print(checkpoint_list)
pt = seed_summary(result_dict, checkpoint_list)

# save to excel
df = pd.DataFrame(pt.rows, columns=pt.field_names)
df.to_excel('output_h36m_s1_canonical_revolute_input_centering.xlsx', index=False)

+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+
|      Subset      | seed0_e1 | seed1_e1 | seed2_e1 | seed3_e1 | seed4_e1 | Average_e1 | seed0_e2 | seed1_e2 | seed2_e2 | seed3_e2 | seed4_e2 | Average_e2 |
+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+
|    3DHP TEST     |  95.84   |  96.22   |  98.10   |  96.01   |  98.27   |   96.89    |  66.67   |  64.71   |  65.84   |  64.46   |  65.95   |   65.53    |
| 3DHP TEST (univ) |  72.62   |  70.95   |  73.98   |  70.46   |  72.91   |   72.18    |  59.51   |  57.73   |  58.74   |  57.53   |  58.83   |   58.47    |
|    3DHP TRAIN    |  86.53   |  86.10   |  86.60   |  87.76   |  89.42   |   87.28    |  65.30   |  64.94   |  64.48   |  65.23   |  66.15   |   65.22    |
|    FIT3D ALL     |  123.04  |  148.19  |  110.21  |  160

# Total

In [ ]:
summary_table = prettytable.PrettyTable()
summary_table.field_names = ['checkpoint', 'subset', 'MPJPE', 'P-MPJPE']
for checkpoint in result_dict.keys():
    for subset in result_dict[checkpoint].keys():
        e1 = result_dict[checkpoint][subset]['e1']
        e2 = result_dict[checkpoint][subset]['e2']
        row = [checkpoint, subset, f'{e1:.2f}', f'{e2:.2f}']
        summary_table.add_row(row)
print(summary_table)

+-------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------+--------+---------+
|                                                                       checkpoint                                                                      |                                          subset                                          | MPJPE  | P-MPJPE |
+-------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------+--------+---------+
|                        MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_adaptive_focal_input_centering_tr_s1                       |     H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXE

In [ ]:
checkpoint_list = [
    "MB_train_h36m_gt_cam_no_factor_with_canonical2",
    'MB_train_h36m_gt_cam_no_factor_with_canonical2_tr_s1_ts_s5678',
    'MB_train_fit3d_gt_cam_no_factor_with_canonical2_tr_s03',
    'MB_train_fit3d_gt_cam_no_factor_with_canonical2_ts_s4710',
]
output_file = f"output_with_canonical2.xlsx"
print_and_save_result(result_dict, checkpoint_list, output_file, save_excel=False)
print()

Checkpoint MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_with_Kvirt에 대한 결과가 없습니다.
+---------------------+--------+--------+---------------------------------------------------------------+--------------------------------------+
|       Dataset       |   E1   |   E2   |                           Checkpoint                          |                Subset                |
+---------------------+--------+--------+---------------------------------------------------------------+--------------------------------------+
| 3DHP-TEST_ALL_TRAIN | 186.13 | 125.26 |     MB_train_fit3d_gt_cam_no_factor_with_canonical2_tr_s03    | 3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN |
| 3DHP-TEST_ALL_TRAIN | 171.22 | 108.59 |    MB_train_fit3d_gt_cam_no_factor_with_canonical2_ts_s4710   | 3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN |
| 3DHP-TEST_ALL_TRAIN | 115.10 | 57.43  |         MB_train_h36m_gt_cam_no_factor_with_canonical2        | 3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN |
| 3DHP-TEST_ALL_TRAIN | 122.79